In [1]:
!git clone https://github.com/UN-GCPDS/BananaRipenessCREDA.git
%cd BananaRipenessCREDA
!pip install -e . -q
%cd ..

Cloning into 'BananaRipenessCREDA'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 247 (delta 119), reused 197 (delta 75), pack-reused 0 (from 0)
Receiving objects: 100% (247/247), 8.30 MiB | 38.30 MiB/s, done.
Resolving deltas: 100% (119/119), done.
/kaggle/working/BananaRipenessCREDA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for banana_creda (pyproject.toml) ... done
/kaggle/working


# Mathematical Formulation of CREDA

The Class-Regularized Entropy Domain Adaptation (CREDA) method aligns feature distributions between the Source and Target domains by maximizing the Mutual Information between them. This is achieved by estimating Rényi’s Quadratic Entropy through the Kernel Trick.

## 1. Kernel Definition and Gram Matrix
Let $X = \{x_1, ..., x_N\}$ be a set of feature vectors (encoder output). The similarity between two samples is computed using a Gaussian (RBF) Kernel:

$$\kappa(x_i, x_j) = \exp\left( - \frac{\|x_i - x_j\|^2}{2\sigma^2} \right)$$
Where $\sigma$ is determined dynamically using the median heuristic over the pairwise distances of the current batch.
The Gram Matrix $K \in \mathbb{R}^{N \times N}$ contains the kernel evaluations $K_{ij} = \kappa(x_i, x_j)$. To treat it as a probability density matrix (similar to a covariance matrix in a Reproducing Kernel Hilbert Space), it is normalized by its trace:

$$A = \frac{K}{\text{tr}(K)}$$

## 2. Rényi’s Quadratic Entropy ($H_2$)
The order-2 entropy ($H_2$) of the distribution represented by matrix $A$ is calculated using the matrix spectrum (eigenvalues $\lambda$). Thanks to the properties of the trace, this simplifies to:

$$H_2(A) = -\log_2\left( \sum_{i=1}^N \lambda_i^2(A) \right) = -\log_2\left( \text{tr}(A^2) \right)$$
This formulation avoids explicit and computationally expensive eigenvalue decomposition, allowing for efficient backpropagation during training.

## 3. Confidence Weighting (Uncertainty Mitigation)
To prevent "negative alignment" caused by noisy samples in the Target domain, a confidence weight $w_j$ is calculated for each target sample $x_j^t$ based on its normalized prediction entropy $\hat{U}$:

$$w_j = 1 - \hat{U}(x_j^t) = 1 - \frac{H(\hat{y}_j)}{\log_2(C)}$$
The Target kernel matrix is then weighted using the Hadamard product ($\odot$) with the weight matrix $W = w w^T$:

$$\tilde{K}^t = K^t \odot W$$

## 4. Joint Matrix and Alignment
To evaluate the discrepancy between domains, a Joint Kernel Matrix ($K_{mix}$) is constructed using a block-matrix structure:

$$K_{mix} = \begin{pmatrix} K^s & K^{st} \\ (K^{st})^T & \tilde{K}^t \end{pmatrix}$$
The alignment objective (CREDA Loss) aims to maximize the Mutual Information, which is equivalent to minimizing the following class-conditional loss function:

$$\mathcal{L}_{align} = \frac{1}{|C|} \sum_{c \in C} \left( H_2(K_{mix}^{(c)}) - \frac{1}{2} \left[ H_2(K_s^{(c)}) + H_2(\tilde{K}_t^{(c)}) \right] \right)$$

## 5. Total Loss Function
The final objective function combines supervised classification on the Source domain with the domain alignment term to ensure robust decision boundaries across both domains:

$$\mathcal{L}_{total} = \mathcal{L}_{CE}(X_s, Y_s) + \lambda_{creda} \mathcal{L}_{align}$$
Where:
$\mathcal{L}_{CE}$: Cross-Entropy Loss (Supervised learning on the Source domain).
$\mathcal{L}_{align}$: CREDA Alignment Loss (Unsupervised domain adaptation).
$\lambda_{creda}$: Regularization hyperparameter that controls the strength of the adaptation.


In [2]:
!python /kaggle/working/BananaRipenessCREDA/scripts/train.py --config /kaggle/working/BananaRipenessCREDA/configs/mobilenetv3_experiment.yaml

2026-02-12 16:56:14.291135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770915374.485521      69 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770915374.544511      69 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770915375.014816      69 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770915375.014865      69 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770915375.014870      69 computation_placer.cc:177] computation placer alr

In [3]:
!python /kaggle/working/BananaRipenessCREDA/scripts/train.py --config /kaggle/working/BananaRipenessCREDA/configs/vit_experiment.yaml

2026-02-12 17:09:28.417279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770916168.440727     205 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770916168.448974     205 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770916168.467466     205 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770916168.467505     205 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770916168.467513     205 computation_placer.cc:177] computation placer alr

In [4]:
!python /kaggle/working/BananaRipenessCREDA/scripts/train.py --config /kaggle/working/BananaRipenessCREDA/configs/efficientnet_experiment.yaml

2026-02-12 18:13:43.431285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770920023.455308     341 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770920023.462586     341 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770920023.481154     341 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770920023.481185     341 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770920023.481188     341 computation_placer.cc:177] computation placer alr

In [5]:
!python /kaggle/working/BananaRipenessCREDA/scripts/train.py --config /kaggle/working/BananaRipenessCREDA/configs/resnet_experiment.yaml

2026-02-12 18:33:05.742547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770921185.765981     477 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770921185.772951     477 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770921185.791153     477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770921185.791184     477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770921185.791189     477 computation_placer.cc:177] computation placer alr